## All Offers Daily Submissions Heatmap

In [1]:
import itertools
import datetime as dt
import psycopg2
import pandas as pd
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go

from nbstyler import DATA_STYLE as DS

plotly.offline.init_notebook_mode(connected=True)

%matplotlib notebook
%matplotlib inline

### Objectives



### Data Preparation

Get the data and prepare the `Pandas.DataFrame` object.

In [2]:
with psycopg2.connect('dbname=jobsbg') as conn:
    daily_df = pd.read_sql_query('SELECT * FROM all_offers.ao_count_daily', conn, index_col='subm_date')

daily_df.index = pd.to_datetime(daily_df.index)
daily_df.head(5)

,subm_count
subm_date,
2017-09-27,1840
2017-09-28,1658
2017-09-29,1741
2017-09-30,12
2017-10-01,36


#### Shaping the dataframe

Since I wanted to shape the heatmap in a way where the most active day of the week (Monday) is on the bottom, and the other days of the week following on top, I have to provide `Plotly` with a list of list with each of the lists being the same day (e.g. Monday) for the whole time period. In order to do that I first filled in the missing dates to create a complete rectangular dataframe table which could then be reshaped. See also: https://stackoverflow.com/a/45850005_

In [3]:
min_ts = min(daily_df.index)
max_ts = max(daily_df.index)
idx = pd.date_range(
    min_ts - dt.timedelta(days=min_ts.weekday()),
    max_ts + dt.timedelta(days=6-max_ts.weekday()))

In [4]:
daily_df = daily_df.reindex(idx)
daily_df.head()

,subm_count
2017-09-25,NaN
2017-09-26,NaN
2017-09-27,1840.0
2017-09-28,1658.0
2017-09-29,1741.0


Now the dataframe can be reshaped into columns representing the days of the week, and then transposed to completely match our heatmap structure. That will make composing the chart itself very easy.

In [5]:
matrix_df_values = pd.DataFrame(daily_df.values.reshape(len(daily_df)//7, 7), columns=daily_df.index[:7].strftime('%A'))
matrix_df_values = matrix_df_values.T
matrix_df_values.head(7)

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
Monday,NaN,2567.0,2630.0,2537.0,2475.0,2457.0,2364.0,2323.0,2303.0,2123.0,...,2444.0,2486.0,2521.0,2371.0,2210.0,2565.0,902.0,742.0,812.0,877.0
Tuesday,NaN,2435.0,1857.0,2002.0,1943.0,1675.0,1781.0,1711.0,1713.0,1618.0,...,1938.0,1776.0,1850.0,1728.0,19.0,1882.0,706.0,647.0,456.0,56.0
Wednesday,1840.0,1740.0,1704.0,1837.0,1688.0,1654.0,1723.0,1666.0,1617.0,1355.0,...,1841.0,1653.0,1664.0,1504.0,2790.0,1657.0,630.0,523.0,604.0,NaN
Thursday,1658.0,1757.0,1688.0,1650.0,1532.0,1533.0,1550.0,1572.0,1444.0,1236.0,...,1499.0,1547.0,1438.0,1524.0,2068.0,1502.0,636.0,468.0,615.0,NaN
Friday,1741.0,1751.0,1570.0,1621.0,1618.0,1592.0,1549.0,1564.0,1340.0,1359.0,...,1660.0,1519.0,1630.0,1504.0,1905.0,940.0,643.0,618.0,623.0,NaN
Saturday,12.0,24.0,41.0,19.0,37.0,27.0,6.0,0.0,28.0,15.0,...,9.0,7.0,13.0,10.0,8.0,26.0,1.0,2.0,2.0,NaN
Sunday,36.0,8.0,11.0,6.0,10.0,1.0,9.0,16.0,6.0,7.0,...,4.0,7.0,7.0,14.0,9.0,0.0,1.0,0.0,1.0,NaN


In [6]:
matrix_df_dates = daily_df
matrix_df_dates = matrix_df_dates.drop(['subm_count'], axis=1).reset_index()
matrix_df_dates = pd.DataFrame(matrix_df_dates.values.reshape(len(matrix_df_dates)//7, 7)).T
matrix_df_dates

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
0,2017-09-25,2017-10-02,2017-10-09,2017-10-16,2017-10-23,2017-10-30,2017-11-06,2017-11-13,2017-11-20,2017-11-27,...,2020-02-03,2020-02-10,2020-02-17,2020-02-24,2020-03-02,2020-03-09,2020-03-16,2020-03-23,2020-03-30,2020-04-06
1,2017-09-26,2017-10-03,2017-10-10,2017-10-17,2017-10-24,2017-10-31,2017-11-07,2017-11-14,2017-11-21,2017-11-28,...,2020-02-04,2020-02-11,2020-02-18,2020-02-25,2020-03-03,2020-03-10,2020-03-17,2020-03-24,2020-03-31,2020-04-07
2,2017-09-27,2017-10-04,2017-10-11,2017-10-18,2017-10-25,2017-11-01,2017-11-08,2017-11-15,2017-11-22,2017-11-29,...,2020-02-05,2020-02-12,2020-02-19,2020-02-26,2020-03-04,2020-03-11,2020-03-18,2020-03-25,2020-04-01,2020-04-08
3,2017-09-28,2017-10-05,2017-10-12,2017-10-19,2017-10-26,2017-11-02,2017-11-09,2017-11-16,2017-11-23,2017-11-30,...,2020-02-06,2020-02-13,2020-02-20,2020-02-27,2020-03-05,2020-03-12,2020-03-19,2020-03-26,2020-04-02,2020-04-09
4,2017-09-29,2017-10-06,2017-10-13,2017-10-20,2017-10-27,2017-11-03,2017-11-10,2017-11-17,2017-11-24,2017-12-01,...,2020-02-07,2020-02-14,2020-02-21,2020-02-28,2020-03-06,2020-03-13,2020-03-20,2020-03-27,2020-04-03,2020-04-10
5,2017-09-30,2017-10-07,2017-10-14,2017-10-21,2017-10-28,2017-11-04,2017-11-11,2017-11-18,2017-11-25,2017-12-02,...,2020-02-08,2020-02-15,2020-02-22,2020-02-29,2020-03-07,2020-03-14,2020-03-21,2020-03-28,2020-04-04,2020-04-11
6,2017-10-01,2017-10-08,2017-10-15,2017-10-22,2017-10-29,2017-11-05,2017-11-12,2017-11-19,2017-11-26,2017-12-03,...,2020-02-09,2020-02-16,2020-02-23,2020-03-01,2020-03-08,2020-03-15,2020-03-22,2020-03-29,2020-04-05,2020-04-12


#### Building the heatmap

The heatmap is composed of three dimensions: x, y, and z. The z dimension will hold our values matrix. Let's prepare that first:

In [7]:
cell_values = [
    [v for v in matrix_df_values.loc['Monday'].values],
    [v for v in matrix_df_values.loc['Tuesday'].values],
    [v for v in matrix_df_values.loc['Wednesday'].values],
    [v for v in matrix_df_values.loc['Thursday'].values],
    [v for v in matrix_df_values.loc['Friday'].values],
    [v for v in matrix_df_values.loc['Saturday'].values],
    [v for v in matrix_df_values.loc['Sunday'].values]
]

The approach to prepare the labels is the same:

In [8]:
cell_labels = [
    [str(v)[:10] for v in matrix_df_dates.iloc[0].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[1].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[2].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[3].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[4].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[5].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[6].values]
]

A manually defined color scale to match with the presentation style:

In [23]:
colorscale =[
    [0.0, DS['colorramp']['acc1'][0]],
    [0.1111111111111111, DS['colorramp']['acc1'][1]],
    [0.2222222222222222, DS['colorramp']['acc1'][2]],
    [0.3333333333333333, DS['colorramp']['acc1'][3]],
    [0.4444444444444444, DS['colorramp']['acc1'][4]],
    [0.5555555555555556, DS['colorramp']['acc1'][5]],
    [0.6666666666666666, DS['colorramp']['acc1'][6]],
    [0.7777777777777778, DS['colorramp']['acc1'][7]],
    [0.8888888888888888, DS['colorramp']['acc1'][8]],
    [1.0, DS['colorramp']['acc1'][9]]
]

Finally, the heatmap trace definition:

In [24]:
hm_trace = go.Heatmap(
    x=[w for w in daily_df.index[::7]],
    y=[d for d in matrix_df_values.index],
    z=cell_values,
    text=cell_labels,
    hoverinfo='text+z+y',
    colorscale=colorscale,
    showscale=False,
    xgap=0,
    ygap=0,
)

data = [hm_trace]

Layout preparation is fairly standard. Labels have been turned off for both axes.

In [25]:
layout = go.Layout(
    paper_bgcolor=DS['colors']['bg1'],
    plot_bgcolor=DS['colors']['bg1'],
    title='Offer Submissions Daily Heatmap',
    titlefont=DS['chart_fonts']['title'],
    font=DS['chart_fonts']['text'],
    autosize=True,
    showlegend=False,
    hidesources=True,
    yaxis=dict(),
)

In [26]:
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='all_offers_daily_submissions_heatmap.html')

In [27]:
# Uncomment the line below to export an HTML version of the chart.
plotly.offline.plot(fig, filename='all_offers_daily_submissions_heatmap.html', show_link=False)

'all_offers_daily_submissions_heatmap.html'

In [14]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)